# Mapping Terms Application

This notebook explores the [Chronicling America API](https://chroniclingamerica.loc.gov/about/api/) for creating a map of the frequency certain terms appear in newspapers for a given time.

In [14]:
import pandas as pd
import requests
from geopy import geocoders
from geopy.extra.rate_limiter import RateLimiter
import plotly.offline as pyo
import plotly.graph_objs as go
import time

start_time = time.time()
pyo.init_notebook_mode(connected=True)
searchTerm = "tent+dog,tent+girl,tent+boy"
fileEmpty=False
List = []
List = searchTerm.split(",")
print(List)
for i in List:
    print(i)
    try:
        df = pd.read_pickle(i+'.pkl')
        print("Pickle found.")
        print(df)
    except FileNotFoundError:
        fileEmpty=True
    if(fileEmpty is True): 
        print("Pickle empty.")
        beginYear = 1850
        endYear = 1860
        pg=1
        endpg=4
        #too much data? I'm just grabbing the first few pages right now
        stop = False
        allData = []
        locs = {}
        pd.set_option('display.max_colwidth', -1)
        while not stop:
    #http://chroniclingamerica.loc.gov/search/pages/results/?andtext={searchTerms}&page={startPage?}&ortext={chronam:booleanOrText?}&year={chronam:year?}&date1={chronam:date?}&date2={chronam:date?}&phrasetext={chronam:phraseText?}&proxText={chronam:proxText?}&proximityValue={chronam:proximityValue?}&format=json"
            url = "http://chroniclingamerica.loc.gov/search/pages/results/?phrasetext={0}&dateFilterType=yearRange&date1={1}&date2={2}&page={3}&format=json".format(searchTerm,beginYear,endYear,pg)
            print(url)
            call = requests.get(url)
            data = call.json()
            df = pd.DataFrame(data["items"])
            nf=pd.DataFrame()
            americanPanda=pd.DataFrame()
            #only want city and date for this application
            nf['date']=pd.to_datetime(df['date'])
            nf['citystate']=df['city'].map(lambda a:a[0])+', '+df['state'].map(lambda a: a[0])
            americanPanda['date']=pd.to_datetime(df['date'])
            americanPanda['location']=df['city'].map(lambda a:a[0])+', '+df['county'].map(lambda a: a[0])+', '+df['state'].map(lambda a: a[0])

            loopCount=len(americanPanda.columns)
            secondPanda=pd.DataFrame()
            pdfList=[]
            textList=[]
            for count, url in enumerate(df['url'], start=1):
                print(count)
                call = requests.get(url)
                data = call.json()
                forframe = pd.DataFrame(data)
                forframe.index= [0,1,2]
                forpanda=pd.DataFrame()
                pdfframe=forframe['pdf']
                textframe=forframe['text']
                pdfList.append((pdfframe[0]))
                textList.append((textframe[0]))
            americanPanda['keywords']=i
            print(americanPanda)
            print("=============")
            americanPanda.to_pickle(i+'.pkl')
            if pg==endpg:
                stop = True
            else:
                pg +=1     
    print("--- %s seconds ---" % (time.time() - start_time))
  


['tent+dog', 'tent+girl', 'tent+boy']
tent+dog
Pickle found.
        date                   location  keywords
0 1853-12-23  Rutland, Rutland, Vermont  tent+dog
1 1858-02-23  NaN                        tent+dog
--- 0.017303943634033203 seconds ---
tent+girl
Pickle found.
         date                      location   keywords
0  1858-12-15  NaN                           tent+girl
1  1860-09-19  Cincinnati, Hamilton, Ohio    tent+girl
2  1852-11-01  New York, New York, New York  tent+girl
3  1854-10-19  New York, New York, New York  tent+girl
4  1854-10-26  New York, New York, New York  tent+girl
5  1854-03-14  New York, New York, New York  tent+girl
6  1855-03-06  New York, New York, New York  tent+girl
7  1854-04-26  New York, New York, New York  tent+girl
8  1854-05-18  New York, New York, New York  tent+girl
9  1858-01-12  New York, New York, New York  tent+girl
10 1855-12-04  New York, New York, New York  tent+girl
11 1850-05-15  New York, New York, New York  tent+girl
12 1852-04-29